# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results.

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating above 4.5?
- How many businesses have a rating below 3?
- Return the text of the oldest review in the table.
- Return the overall rating of the business with the oldest review. 
- Find the highest rated business and return text of the newest review of the three.
- Find the lowest rated business and return text of the newest review of the three.  


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [9]:
## Connect to DB server on AWS
import mysql.connector
from mysql.connector import errorcode
import config
db_name= 'yelp_businesses'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    
)
cursor = cnx.cursor()
print(cnx)

In [10]:
## Create new DB 
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [ ]:
# Create a table for the Businesses


In [ ]:
# Create a table for the reviews

## Part 2: Create ETL pipeline for the business data from the API

In [11]:
# write a function to make a call to the API
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)
keys = get_keys("/Users/flatironschool/.secret/yelp_api.json") 

api_key = keys['api_key']

print(api_key)

kFSOsZc9eAFWgYmkrfg76jE1NPHVB__yEmFR7SCnMg0nsp7inQ9etProZ1m4mEY5m5Vm0jWuuZVvN1kHdiTz2QJTtNPR1j3CQLLpiwDpioMpcEvNQ7PdZIkRUv45XXYx


In [14]:
# import requests
# term = 'Mexican'
# location = 'Astoria NY'
# SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
# print(response)
# print(type(response.text))
# print(response.text[:1000])
data=response.json()
data.keys()


dict_keys(['businesses', 'total', 'region'])

In [15]:
# Your code here; use a function or loop to retrieve all the results from your original request
response = requests.get(url, headers=headers, params=url_params)

data=response.json()


import time
import requests

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key)) #can parse before adding to results database instead of parsing after results db gets created  
        time.sleep(1) #Wait a second
        cur += 50
    return results

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))


1000 total matches found.
20


In [23]:
# data = df
# print(len(data))
# print(len(data[0]))
# data[0][0].keys()

print(data[0][0].keys())

# for i in data:
#     for x in i:
#         try:
#             print(x['id'])
#         except:
#             pass




dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance'])


In [6]:
# write a function to parse the API response 
response.json().keys()
# for key in response.json().keys():
#     print(key)
#     value = response.json()[key] #Use standard dictionary formatting
#     print(type(value)) #What type is it?
#     print('\n\n') #Separate out data
#response.json()['businesses'][:2]
import pandas as pd

# df = pd.DataFrame.from_dict(response.json()['businesses'])
# print(len(df)) #Print how many rows
# print(df.columns) #Print column names
# df.head()
type(response.json())


dict

In [7]:
def find_name(b):
    return [x['name'] for i in b for x in i]

def find_rating(b):
    return [x['rating'] for i in b for x in i]

def find_price(b):
    lst = []
    for i in b:
        for x in i:
            try:
                lst.append(x['price'])
            except:
                lst.append('na')
    return lst

print(type(find_rating(data)[0]))


def find_id(b):
    return [x['id'] for i in b for x in i]

business_ids = find_id(data)
# bus_ids = business_ids[:10]
# bus_ids


            




<class 'float'>


In [8]:
# add a dictionary for all the values
businesses = []
def create_businesses(d):
    businesses = []
    for i in d:
        for x in i:
            try:
                bus = {'name': x['name'], 
                      'rating': x['rating'],
                      'price': x['price'],
                      'bus_id': x['id']}
            except:
                bus = {'name': x['name'], 
                      'rating': x['rating'],
                      'price': 'na',
                      'bus_id': x['id']}
                
            if bus not in businesses:
                businesses.append(bus)
    return businesses


create_businesses(data)
print(len(create_businesses(data)))

bus_table = create_businesses(data)



700


In [9]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }

def yelp_review_call(url_params, api_key, key):  ## why did we use key and add to url ? 
    url = f"https://api.yelp.com/v3/businesses/{key}/reviews"
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['reviews']
    for n in data: # replacing id with the actual key or bus_id  ##confusedddd
        for k, v in n.items():
            if k == 'id':
                n[k] = key
    return data

review_list = []#[yelp_call(url_params, api_key, key=str(i)) for i in business_ids]
count = 0
for i in business_ids:
    review_list.append(yelp_review_call(url_params, api_key, key=str(i)))
    count += 1
    print(count)

# def all_results(review_list):
#     num = response.json()['total']
#     print('{} total matches found.'.format(num))
#     cur = 0
#     results = []
#     while cur < num and cur < 1000:
#         url_params['offset'] = cur
#         results.append(review_list) #can parse before adding to results database instead of parsing after results db gets created  
#         time.sleep(1) #Wait a second
#         cur += 50
#     return results



# dataf = all_results(review_list)
dataf = review_list

print(len(dataf))


type(dataf)
dataf[0]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


[{'id': 'hB2S1y5T9ufMz5ksHHhdIA',
  'url': 'https://www.yelp.com/biz/rizzos-fine-pizza-astoria?adjust_creative=Wx5NL3v6AeuprywNIz0KNA&hrid=zSF3h6k6wevHtcvJN4_Nig&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=Wx5NL3v6AeuprywNIz0KNA',
  'text': "This five star was the most no-brainer five stars I've ever left. The folks are so kind, the place is clean and the pizza is all through the roof.\n\nI got a...",
  'rating': 5,
  'time_created': '2019-06-12 09:15:57',
  'user': {'id': '9Pk0qHnLDJ1GEeJar2UfeQ',
   'profile_url': 'https://www.yelp.com/user_details?userid=9Pk0qHnLDJ1GEeJar2UfeQ',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/GgkcTpSbq5vgjlbAHtprFQ/o.jpg',
   'name': 'Wesley B.'}},
 {'id': 'hB2S1y5T9ufMz5ksHHhdIA',
  'url': 'https://www.yelp.com/biz/rizzos-fine-pizza-astoria?adjust_creative=Wx5NL3v6AeuprywNIz0KNA&hrid=8WWPf8mBJUchBJMP9EBO2A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=Wx5NL3v6AeuprywNIz0KNA',
  'text': "Ri

In [10]:
type(dataf[0][0])

dict

In [11]:
if dataf[0][0]['id'] in business_ids:
    print(True)
else:
    print(False)

True


In [12]:

def find_review(d):
#     lst = []
#     for i in d:
#         for x in i:
#             for z in x:
#                 lst.append(z['text'].replace('\n', ''))
#     return lst
    return [z['text'].replace('\n', '') 
            for i in d 
            for x in i 
            for z in x]

#find_review(dataf)

def find_user_id(d):
#     lst = []
#     for i in d:
#         for x in i:
#             for z in x:
#                 lst.append(z['user']['id'])
#     return lst
    return [x['user']['id'] 
            for i in d 
            for x in i]

print(len(find_user_id(dataf)))

def find_time(d):
#     lst = []
#     for i in d:
#         for x in i:
#             for z in x:
#                 lst.append(z['time_created'])
#     return lst
    return [z['time_created']
           for i in d
           for x in i
           for z in x]

#find_time(dataf)
      
def find_bus_id(d):
    return [x['id'] 
            for i in d 
            for x in i]

print(len(find_bus_id(dataf)))
    
from collections import Counter
Counter(find_bus_id(dataf)).most_common()
                

2038
2038


[('hB2S1y5T9ufMz5ksHHhdIA', 3),
 ('1OZPOWZwJr-hpvMt6TD4ug', 3),
 ('KjlNK_o3kWtsjsIx6ZgF0A', 3),
 ('JjpCF17AxF6RPOyimw21cw', 3),
 ('5Uo_a_Kl53O_yKglSR5nBA', 3),
 ('Lw58gDv9ipwtOcg2EIJp3w', 3),
 ('mSUN6AosDrAuM9H29qEhgg', 3),
 ('4q8jF2pKoWYbJXYmSNMG1g', 3),
 ('NBeQAgTo9hlJxgp9xbMO-w', 3),
 ('Sw7LUCN0Qo3V3D5xoqY6hg', 3),
 ('kWTvmbZtcU6Gc2NuYNO4dA', 3),
 ('p-BHJdQ57eM-WlKTP7B44w', 3),
 ('ehr6yuRGlszujcDnlXhnMg', 3),
 ('pFPiqFLeL-peQdVSfPj_lg', 3),
 ('iJg_48scQgvK3fEm47fcIw', 3),
 ('rhCs1nF5KTNwx-4YeyoK3g', 3),
 ('UlQK0jkNEw3j9-_S9r4Rgw', 3),
 ('-Zebvg7OAy1YgWRwIw7Ogg', 3),
 ('TFdLbSBwuIjikxBl4vvkQQ', 3),
 ('cEWF70xMbOvOhr4CDkroOw', 3),
 ('24EmbJBGGGnbAOQEJljrrg', 3),
 ('p913iM6bsB0JvIjaUJZUkw', 3),
 ('TkrV0mQu38XtbbvhZmiw5A', 3),
 ('JLiEQTHp0hTE0TVnZNSDIg', 3),
 ('gP8BPFNO0F1ixR7OTlGJNw', 3),
 ('VW24_7r4oZz-i4V_8VhrQw', 3),
 ('r2MaSxP0J07ZfF6ze1KaPw', 3),
 ('S06na3QRjSQfVX-lH05xEg', 3),
 ('1ZKbUeFAs0ltSnAYq4ELKw', 3),
 ('qey4HXKGJTlhAcN5ZbEdZQ', 3),
 ('vCMRKyKkVV1xfH0GwBLdtA', 3),
 ('-0fjW

In [13]:
def create_reviews(d):
    lst = []
    for i in d:
        for z in i:
            rev = {'user_id': z['user']['id'],
                  'review': z['text'].replace('\n', ''),
                  'time': z['time_created'],
                  'bus_id': z['id']}
            lst.append(rev)
    return lst

rev_table = create_reviews(dataf)
rev_table
# print(len(rev_table))


[{'user_id': '9Pk0qHnLDJ1GEeJar2UfeQ',
  'review': "This five star was the most no-brainer five stars I've ever left. The folks are so kind, the place is clean and the pizza is all through the roof.I got a...",
  'time': '2019-06-12 09:15:57',
  'bus_id': 'hB2S1y5T9ufMz5ksHHhdIA'},
 {'user_id': 'jt0XbD-gyEGIxL086p7QOw',
  'review': "Rizzo's provided incredible customer service. My friends and I ordered Rizzo's for a friend's' surprise birthday party and the original ETA for the pizza...",
  'time': '2019-04-27 10:11:00',
  'bus_id': 'hB2S1y5T9ufMz5ksHHhdIA'},
 {'user_id': '0G-QF457q_0Z_jKqh6xWiA',
  'review': "Rizzo's is a dream come true for a vegan gal like me, whose fondest childhood memories include going to the local old school pizzeria in Ozone Park with her...",
  'time': '2019-03-14 07:52:02',
  'bus_id': 'hB2S1y5T9ufMz5ksHHhdIA'},
 {'user_id': 'f4QTgUdl0-DOsucgGOeDyw',
  'review': "Went to a place called Milk Flower the other day.. It's located in Astoria.. Milk flower of cour

In [187]:
# from collections import Counter
# my_list = [ids['bus_id'] for ids in rev_table]
# Counter(my_list).most_common()

# unique = []
# for i in rev_table:
#     if i not in unique:
#         unique.append(i)
    
# #print(len(unique))
# rev_table = unique
    

## Part 3: Create ETL pipeline for the review data from the API

In [14]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant
# name': x['name'], 
#                       'rating': x['rating'],
#                       'price': x['price'],
#                       'bus_id'
db_name= 'yelp_businesses'

TABLES = {}
TABLES['businesses'] = (
    "CREATE TABLE businesses ("
    "  name varchar(100) NOT NULL,"    # these are also just commands but tables havent been created yet, these are sql commands in python, are just strings 
    "  rating float(2) NOT NULL,"
    "  price varchar(5) NOT NULL,"
    "  bus_id varchar(30) NOT NULL,"
    "  PRIMARY KEY (bus_id)"
    ") ENGINE=InnoDB")

#  {'user_id': z['user']['id'],
#                           'review': z['text'].replace('\n', ''),
#                           'time': z['time_created'],
#                           'bus_id': z['id']}

TABLES['reviews'] = (
    "CREATE TABLE reviews ("
    "  review text,"
    "  time varchar(40) NOT NULL,"
    "  bus_id varchar(30) NOT NULL,"
    "  user_id varchar(30) NOT NULL"
    ") ENGINE=InnoDB")



In [15]:
TABLES['businesses']

'CREATE TABLE businesses (  name varchar(100) NOT NULL,  rating float(2) NOT NULL,  price varchar(5) NOT NULL,  bus_id varchar(30) NOT NULL,  PRIMARY KEY (bus_id)) ENGINE=InnoDB'

In [16]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database= db_name
)
cursor = cnx.cursor()
print(cnx)
# write a function that take a business id 
# and makes a call to the API for reivews
# then parse out the relevant information

In [17]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='') # here we executing and actually creating the tables 
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

# write a function to insert the parsed data into the reviews table

Creating table businesses: already exists.
Creating table reviews: OK


In [18]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = db_name
)
cursor = cnx.cursor()


In [57]:
## Part 4: Write SQL queries that will answer the questions posed. 

In [19]:
from datetime import date, datetime, timedelta

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = db_name
)
cursor = cnx.cursor()
#   "CREATE TABLE businesses ("
#     "  name varchar(100) NOT NULL,"
#     "  rating float(2) NOT NULL,"
#     "  price varchar(5) NOT NULL,"
#     "  bus_id varchar(30) NOT NULL,"
#     "  PRIMARY KEY (bus_id)"
#     ") ENGINE=InnoDB")

#  "CREATE TABLE reviews ("
#     "  review text,"
#     "  time varchar(40) NOT NULL,"
#     "  bus_id varchar(30) NOT NULL,"
#     "  user_id varchar(30) NOT NULL,"
#     "  PRIMARY KEY (bus_id), UNIQUE KEY user_id (user_id)"
#     ") ENGINE=InnoDB")

add_businesses = ("INSERT INTO businesses "
               "(name, rating, price, bus_id) " # these are just commands 
               "VALUES (%s, %s, %s, %s)")
add_reviews = ("INSERT INTO reviews "
              "(review, time, bus_id, user_id) "
              "VALUES (%s, %s, %s, %s)")


In [20]:
bus_tup = [(i['name'], i['rating'], i['price'], i['bus_id']) for i in bus_table]
rev_tup = [(i['review'], i['time'], i['bus_id'], i['user_id']) for i in rev_table] #executing values into the tables from the functions we created earlier 

# for i in bus_tup:
#     try:
#         cursor.execute(add_businesses, i)
#     except:
#         pass

for i in rev_tup:
    cursor.execute(add_reviews, i)
cnx.commit()


###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key)) #can parse before adding to results database instead of parsing after results db gets created  
        time.sleep(1) #Wait a second
        cur += 50
    return results

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id VARCHAR(16) AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```